In [1]:
from typing import Union

from p1_perceptron import Perceptron, PerceptronLayer, PerceptronNetwork, step_activation

## Test Perceptron

Initialiseer een `Perceptron` voor elk van de INVERT-, AND- en OR-poorten en test of ze op de juiste manier werken.

### AND-poort

Een AND-poort activeert alleen als beide input op `1` staan. Beide inputs tellen even zwaar mee. Daarom maken we alle
`weights` gewoon 1 en de `threshold` de som van de weights, dus `2`.

In [2]:
AND: Perceptron = Perceptron([1, 1], -2, step_activation)

print(AND)

<Perceptron {activation: step, bias: -2, weights: [1, 1]}>


Nu kunnen we een lijst maken aan alle mogelijke combinaties van input en de verwachte output.

In [3]:
truth_table_AND = (
    ([0, 1], 0),
    ([1, 0], 0),
    ([1, 1], 1),
    ([0, 0], 0),
)

Nu kunnen we een functie schrijven om de output van de perceptron tegen de waarheidstabel te vergelijken. 

In [4]:
def compare_to_truth_table(table: tuple[list[Union[float, int]], list[int]], 
                           obj: Union[Perceptron, PerceptronNetwork]
                           ) -> None:
    for row in table:
        output = obj.activate(row[0]) if isinstance(obj, Perceptron) else obj.predict(row[0])
        print(f"Input {row[0]} geeft als output: {output}")

        assert output == row[1]

En deze vervolgens te gebruiken.

In [5]:
compare_to_truth_table(truth_table_AND, AND)

Input [0, 1] geeft als output: 0.0
Input [1, 0] geeft als output: 0.0
Input [1, 1] geeft als output: 1.0
Input [0, 0] geeft als output: 0.0


### OR-poort

Een OR-poort activeert als minstens 1 van de inputs op `1` staan en als we de threshold de helft van de som van de 
`weights` maken dan zal dit het geval zijn.

In [6]:
OR = Perceptron((1, 1), -1, step_activation)
print(OR)

<Perceptron {activation: step, bias: -1, weights: (1, 1)}>


En vervolgens de truth table van OR te definiëren de we vergelijken met de outputs van de perceptron.

In [7]:
truth_table_OR = (
    ([0, 0], 0),
    ([0, 1], 1),
    ([1, 0], 1),
    ([1, 1], 1),
)

compare_to_truth_table(truth_table_OR, OR)

Input [0, 0] geeft als output: 0.0
Input [0, 1] geeft als output: 1.0
Input [1, 0] geeft als output: 1.0
Input [1, 1] geeft als output: 1.0


### INVERT-poort

Een INVERT-poort draait simpelweg de waarde van de input om, dus 0 wordt 1 en 1 wordt 0. Vertaald naar de perceptron is
het dus: geef `1` terug als de input `0` is. Dit kunnen we doen door de threshold negatief, dus `-1`, en de 
weight `0` te maken.

In [8]:
truth_table_INVERT = (
    ([0], 1),
    ([1], 0),
)

INVERT = Perceptron((-1,), 0, step_activation)
print(INVERT)

<Perceptron {activation: step, bias: 0, weights: (-1,)}>


In [9]:
compare_to_truth_table(truth_table_INVERT, INVERT)

Input [0] geeft als output: 1.0
Input [1] geeft als output: 0.0


### NOR-poort

Een NOR-gate activeert wanneer alle inputwaardes 0 zijn en in dit geval zijn er 3 inputwaardes. Als we alle gewichten op
`-1` en de threshold op `0` te zetten zal er bij 3 nullen een 1 uitkomen want: $-1*0+-1*0+-1*0+0 = 0$. 

In [10]:
truth_table_NOR = (
    ([0, 0, 0], 1),
    ([0, 0, 1], 0),
    ([0, 1, 0], 0),
    ([0, 1, 1], 0),
    ([1, 0, 0], 0),
    ([1, 0, 1], 0),
    ([1, 1, 1], 0),
)

NOR = Perceptron([-1, -1, -1], 0, step_activation)
print(NOR)

<Perceptron {activation: step, bias: 0, weights: [-1, -1, -1]}>


In [11]:
compare_to_truth_table(truth_table_NOR, NOR)

Input [0, 0, 0] geeft als output: 1.0
Input [0, 0, 1] geeft als output: 0.0
Input [0, 1, 0] geeft als output: 0.0
Input [0, 1, 1] geeft als output: 0.0
Input [1, 0, 0] geeft als output: 0.0
Input [1, 0, 1] geeft als output: 0.0
Input [1, 1, 1] geeft als output: 0.0


### Uitgebreide beslissysteem

Voor een uitgebreide beslissysteem kiezen we 4 willekeurige weights: `[0.1, 0.22, 0.5, 2.1]`. We kiezen ook een 
willekeurige threshold: `0.8`.

In [12]:
perceptron = Perceptron([0.1, 0.22, 0.5, 2.1], -0.8, step_activation)
print(perceptron)

<Perceptron {activation: step, bias: -0.8, weights: [0.1, 0.22, 0.5, 2.1]}>


Om deze perceptron te testen hebben we een aantal willekeurige inputs en hun verwachte outputs.

In [13]:
truth_table_EXPANDED = (
    ([1, 1, 1, 1], 1),
    ([0, 0, 0, 0], 0),
    ([5, 0.1, 2, -1], 0),
    ([5, 5, 5, 5], 1),
    ([8, 0, 0, 0], 1),
)

compare_to_truth_table(truth_table_EXPANDED, perceptron)

Input [1, 1, 1, 1] geeft als output: 1.0
Input [0, 0, 0, 0] geeft als output: 0.0
Input [5, 0.1, 2, -1] geeft als output: 0.0
Input [5, 5, 5, 5] geeft als output: 1.0
Input [8, 0, 0, 0] geeft als output: 1.0


## Test PerceptronNetwork

### XOR-poort

Een XOR-poort heeft 2 inputs en zal `1` teruggeven als één van de 2 inputwaardes `1` is maar niet beide. Dit kunnen we 
doen door eerst een AND-poort en een NOR-poort te checken of de inputs beide positief of negatief zijn. Vervolgens
gebruiken we een OR-poort om de twee outputs naar 1 output te veranderen. Dit betekent dat als beide inputs anders zijn
dat er een `0` uit komt en als beide inputs hetzelfde zijn dat er een 1 uit komt. Dit is precies het omgekeerde van wat 
we willen dus kunnen we dit omdraaien met een INVERT-poort.

Ik heb zelf geprobeerd te beredeneren welke gates ik moest gebruiken i.p.v. het opzoeken van een schema op het internet.

In [14]:
truth_table_XOR = (
    ([0, 0], 0),
    ([0, 1], 1),
    ([1, 0], 1),
    ([1, 1], 0),
)

network = PerceptronNetwork([
    PerceptronLayer([AND, NOR]),
    PerceptronLayer([OR]),
    PerceptronLayer([INVERT]),
])

print(network)

<PerceptronNetwork {
<PerceptronLayer {
	0: <Perceptron {activation: step, bias: -2, weights: [1, 1]}>,
	1: <Perceptron {activation: step, bias: 0, weights: [-1, -1, -1]}>,
}>,
<PerceptronLayer {
	0: <Perceptron {activation: step, bias: -1, weights: (1, 1)}>,
}>,
<PerceptronLayer {
	0: <Perceptron {activation: step, bias: 0, weights: (-1,)}>,
}>,
}>


In [15]:
for row in truth_table_XOR:
    output = network.predict(row[0])

    print(f"Input {row[0]} geeft als output: {output}")

    assert output == [row[1]]

Input [0, 0] geeft als output: [0.0]
Input [0, 1] geeft als output: [1.0]
Input [1, 0] geeft als output: [1.0]
Input [1, 1] geeft als output: [0.0]


### Half-adder

De half-adder neemt twee inputs en geeft 2 outputs terug, de som en rest. In in binaire getallen met 2 inputs is dat 
nogal simpel want $1 + 1 = 10$. Nu kan je dat simpelweg doen met een AND- en een XOR-poort. Voor een XOR-poort hebben we
o.a. een NAND-poort nodig.

In [16]:
NAND = Perceptron([-1.1, -1.1], 2, step_activation)
print(NAND)

<Perceptron {activation: step, bias: 2, weights: [-1.1, -1.1]}>


In [17]:

truth_table_NAND = (
    ([0, 1], 1),
    ([1, 0], 1),
    ([1, 1], 0),
    ([0, 0], 1),
)
compare_to_truth_table(truth_table_NAND, NAND)

Input [0, 1] geeft als output: 1.0
Input [1, 0] geeft als output: 1.0
Input [1, 1] geeft als output: 0.0
Input [0, 0] geeft als output: 1.0


Nu kunnen we de NAND- samen met een OR-poort als het begin van de XOR in de eerste laag stoppen. De carry is gewoon een 
AND-poort. Op de tweede laag maken we de XOR af d.m.v. een AND-poort toe te voegen die het gewicht van de vorige AND op
0 heeft staan. Als laatste voegen we een OR-poort toe die de gewichten van de NAND- en OR-poorten op 0 heeft staan.

In [18]:
truth_table_half_adder = (
    ([0, 0], [0, 0]),
    ([0, 1], [0, 1]),
    ([1, 0], [0, 1]),
    ([1, 1], [1, 0]),
)

half_adder = PerceptronNetwork([
    PerceptronLayer([AND, NAND, OR]),
    PerceptronLayer([Perceptron([1, 0, 0], -1, step_activation), Perceptron([0 , 1, 1], -2, step_activation)]),
])

print(half_adder)

<PerceptronNetwork {
<PerceptronLayer {
	0: <Perceptron {activation: step, bias: -2, weights: [1, 1]}>,
	1: <Perceptron {activation: step, bias: 2, weights: [-1.1, -1.1]}>,
	2: <Perceptron {activation: step, bias: -1, weights: (1, 1)}>,
}>,
<PerceptronLayer {
	0: <Perceptron {activation: step, bias: -1, weights: [1, 0, 0]}>,
	1: <Perceptron {activation: step, bias: -2, weights: [0, 1, 1]}>,
}>,
}>


In [19]:
compare_to_truth_table(truth_table_half_adder, half_adder)

Input [0, 0] geeft als output: [0.0, 0.0]
Input [0, 1] geeft als output: [0.0, 1.0]
Input [1, 0] geeft als output: [0.0, 1.0]
Input [1, 1] geeft als output: [1.0, 0.0]
